In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("F:/simplilearn/Analytics vidya/train_3.csv")

In [3]:
df.head()

,ID,Title,Domain
0,1,"What is good in a decision tree, a large or a ...",Techniques
1,2,Training data only contains single positive label,Techniques
2,3,Calculating percentage contribution of a negat...,Techniques
3,4,Unable to open solution checker!,Hackathons
4,5,User Name Change,Misc


In [4]:
# viewing some texts from the dataframe

for i in range(5):
    print(df.sample(frac=1).iloc[i,1],'  :-  ' , df.iloc[i,2] ,'\n')

Multiple Integer Optimization with weighted / priority approaches   :-   Techniques 

Gradient Boosting Classifier in Python - Unable to run   :-   Techniques 

Career switch from mechanical engineer role to analytics   :-   Techniques 

How to Cluster graphs   :-   Hackathons 

What should I consider while adding or deleting features?   :-   Misc 



In [5]:
df.isna().sum()

ID         0
Title     11
Domain     0
dtype: int64

In [6]:
df.dropna(subset='Title',inplace=True)

In [7]:
df.shape

(3834, 3)

In [8]:
#  some texts have "?"mark or may be other non alphanumeric character attached with swords like "error?" .
# so that NLP gets clear words for text processing and balcncing dataset with synonyms
# we are using NLPAUG for balancing dataset , so NLPAUG may not understand "error?" , so we replace it with "error ?"
# just adding a space between the word and "?"mark

import re

def add_space(text):
    pattern = r"(\w+)\s*(\W)"

    # Replace the matched pattern with a space between the two groups
    replaced_text = re.sub(pattern, r"\1 ", text)

    return replaced_text

In [9]:
df['Title'] = df['Title'].apply(lambda x : add_space(str(x)))

In [10]:
df.shape

(3834, 3)

In [11]:
df['Domain'].value_counts()

Domain
Techniques    1852
Tools          917
Career         437
Hackathons     262
Resources      170
Other          122
Misc            74
Name: count, dtype: int64

In [12]:
# label encoding the target cloumn which has 7 classes

from sklearn.preprocessing import LabelEncoder
df['Domain'] = LabelEncoder().fit_transform(df['Domain'])

### we have many echniques to balanced text dataset . 
### firstly , translating the sentence to other language and then back trnslating it to english . the problem with this is the final translation is exactly same as the orignal sentence , so this is not found useful.
### secondly , replacing some words in the sentence with other worsds based on the context of the sentence . but this didnt give good accuracy.
### third , replace the words in sentence with their synonyms . this  is found good for this problem . as the accuracies were good on both training and test dataset . so i used this . this is done using NLPAUG library
### also thereare other techniques like adding random letters or words to the sentence but that is found not good.

# synonym AUG

In [1]:
import nlpaug.augmenter.word as naw
from tqdm.auto import tqdm # used to know the status of execution
import multiprocessing  #this library is used to include all cores in the process,so that it is done fast .same as n_jobs = -1
import time  # this is just to note the total time taken for the process

In [2]:
augmenter = naw.SynonymAug()

In [19]:
print(df.Title[9],'\n')
print(augmenter.augment(df.Title[9]))

Which algorithm to choose when the response is weak  

['Which algorithm to choose when the reaction is feeble']


In [20]:
# created these to append the  newly generated sentences and later we can create a new balanced data frame with these list
texts_ , labels_ = [],[]

# sometimes this process interrupts , just re-run this cell . sometimes you need to re-run this twice or thrice

time.sleep(1) # adding the timegap of 1 second
start_ = time.time() # noting the start time

# from line 11 to last in this cell .
# line 27 :  , considering the length of unique classes in target variable.
# the highest vallue counts are for class are for "Techniques" 1852  total ,so we are not considering the texts of
# that class , as we are bringing value counts of other class to "techniques" 1852 .
# so we are not  generating new sentences for "techniques"
# line 28 : we have value counts for class "tools" = 917 . so we are doing ( 1852 - 917 ) // 917 ,
# now we have an INT value for how many number of new texts we want to generate for class "tools" ,
# if the value is 2 then  we generate 2 new sentences for each sentence of class "tools" .
# to make its value count closer or equal to counts of "techniques" . same with other classes .
# line 29 : we are just printing what is the ( 1852 - 917 ) // 917 value for each class
# line 31 : consindering the range one particular class (example range oftotal number of texts belonging to class "tools")
# line 32 : calling every text in it using for loop and saving it in object "sentence_"
# line 33 : generating new sentences by replacing words with their synonyms , 
# here "n_" is the number of new texts we want to generate for the given text , ( 1852 - 917 ) // 917 , this value
# num_threads is number of cores to use
# line 35 to 39 : appending each of total generated texts with respective classes to lists texts_ and labels_
# lastly printing the total time taken for the process

for i in df.Domain.value_counts().index.values[1:]:
    n_ = round((1852 - len(df[df['Domain']==i]))/len(df[df['Domain']==i]))
    print(n_ , len(df[df['Domain']==i]))

    for j in tqdm(range(len(df[df['Domain']==i]))):
        sentence_ = df[df['Domain']==i].iloc[j,1]
        aug_texts = augmenter.augment(sentence_ , n = n_,num_thread=multiprocessing.cpu_count())
        
        for k in aug_texts:
            texts_.append(k)
            labels_.append(df[df['Domain']==i].iloc[j,2])
            
print("total time taken to complete is : ",(time.time() - start_)/60 ," minutes .")

1 917


  0%|          | 0/917 [00:00<?, ?it/s]

3 437


  0%|          | 0/437 [00:00<?, ?it/s]

6 262


...

14 122


  0%|          | 0/122 [00:00<?, ?it/s]

24 74


  0%|          | 0/74 [00:00<?, ?it/s]

total time taken to complete is :  0.7961856087048849  minutes .


In [21]:
# from the above process we have created the newly genrated texts , now we combine the original and new genrated texts.

texts_ = texts_ + df.Title.tolist()
labels_ = labels_ + df.Domain.tolist()

In [22]:
# create a new dataset
df_new = pd.DataFrame({"Title" : texts_ , "Domain" : labels_})

In [23]:
df_new.head()

,Title,Domain
0,Plot on Google Map in Python,6
1,R code throws the computer error Error in run ...,6
2,How to do grouping by in Roentgen,6
3,How to take records containing sure name in th...,6
4,Deterrent example of XGBoost utilise Julia on ...,6


In [24]:
df_new["Domain"].value_counts()

Domain
4    1870
5    1852
2    1850
6    1834
1    1834
3    1830
0    1748
Name: count, dtype: int64

In [ ]:
# saving the balanced datafrme , so that the process can be resumed from her ewithout executing the above lines again and again
df_new.to_csv("df_naw.SynonymAug")

In [2]:
# loading the saved csv
df_new = pd.read_csv("df_ContextualWordEmbsAug")

# tfidf

In [3]:
# converting the  texts into vector uing tfidf
# calculating the similarities among vectors using linear kernel

from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer as tf
tf_ = tf(stop_words='english')
tf_.fit(df_new['Title'])
x = tf_.transform(df_new["Title"])
x = linear_kernel(x,x)

In [4]:
x.shape

(12818, 12818)

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,df_new['Domain'] ,test_size = 0.3 , random_state = 12345 , stratify = df_new['Domain'])

# f1_score

In [6]:
# the problem said the f1_Score with average as "macro" should be used as metric for auuracy
# f1_score can be directly used  like we use sklearn.metric.acuracy

from sklearn.metrics import f1_score

# SVM

In [9]:
from sklearn import svm
import os
import multiprocessing

# use all cores while training so that the process is done fast using all cores
os.environ['OMP_NUM_THREADS'] = str(multiprocessing.cpu_count())

In [114]:
svm_model = svm.SVC(random_state=123)
svm_model.fit(x_train,y_train)

SVC(random_state=123)

In [115]:
y_pred = svm_model.predict(x_test)

In [116]:
# printing the f1_score on training and testing set
print(f1_score(y_test,y_pred,average="macro"))

0.717606803215249


In [ ]:
print(f1_score(y_train,svm_model.predict(x_train),average="macro"))

# classification report

In [91]:
from sklearn.metrics import classification_report as cr

print(cr(y_test,y_pred))

# SGD

In [67]:
from sklearn.linear_model import SGDClassifier as sgd

In [93]:
f1_scores = [0.76, 0.70, 0.77, 0.70, 0.69, 0.55, 0.61]
class_weights = {i: 1/f1_scores[i] for i in range(len(f1_scores))}

In [95]:
sgd_ = sgd(random_state = 123 , alpha= 0.0001, loss = 'hinge',penalty = 'l2',class_weight = class_weights) 

In [96]:
sgd_.fit(x_train,y_train)

SGDClassifier(class_weight={0: 1.3157894736842106, 1: 1.4285714285714286,
                            2: 1.2987012987012987, 3: 1.4285714285714286,
                            4: 1.4492753623188408, 5: 1.8181818181818181,
                            6: 1.639344262295082},
              random_state=123)

In [97]:
y_pred = sgd_.predict(x_test)

In [98]:
print(f1_score(y_test,y_pred,average="macro"))

0.707148261237497


In [99]:
print(f1_score(y_train,sgd_.predict(x_train),average="macro"))

0.9046794941290066


In [92]:
# printing classification report
print(cr(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.75      0.77      0.76       525
           1       0.73      0.67      0.70       550
           2       0.75      0.80      0.77       555
           3       0.68      0.72      0.70       549
           4       0.67      0.72      0.69       561
           5       0.56      0.53      0.55       556
           6       0.64      0.57      0.61       550

    accuracy                           0.68      3846
   macro avg       0.68      0.68      0.68      3846
weighted avg       0.68      0.68      0.68      3846



# below are the f1_Scores for each model for test and train data with different parameters , # first test accuracy , second train accuracy

### SGD_ = SGD(random_state = 123 , 'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l2') # 85 , 95

### svm_model_ = svm.svc({'C': 1.0,'break_ties': False, 'cache_size': 200,'class_weight': None,'coef0': 0.0,'decision_function_shape': 'ovr','degree': 3,'gamma': 'scale','kernel': 'rbf','max_iter': -1,'probability': False,'random_state': 123,'shrinking': True,'tol': 0.001,'verbose': False}) # 85 , 94

### svm_model = svm.SVC(random_state=123 ,C = 1 , kernel = 'rbf' , gamma = "scale") # 85 , 94

### svm_model = svm.SVC(random_state=123 ,C = 3 , kernel = 'rbf' , gamma = "scale")# 89 , 99

### svm_model = svm.SVC(random_state=123 ,C = 1 , kernel = 'linear' , gamma = "scale") # 86 , 97

### svm_model = svm.SVC(random_state=123 ,C = 2 , kernel = 'rbf' , gamma = "scale") # 88 , 98

### svm_model = svm.SVC(random_state=123 ,C = 1 , kernel = 'linear' , gamma = "auto") # 86 , 97

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB ,MultinomialNB,BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier , ExtraTreesClassifier , BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score , confusion_matrix
import time

In [9]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)

<IPython.core.display.Javascript object>

In [10]:
# not including svm as its taking long time
clfs = {'KN' : knc, 'NB': mnb,'DT': dtc, 'LR': lrc, 'RF': rfc, 'AdaBoost': abc, 'Bc': bc, 'ETC': etc,'GBDT':gbdt,'xgb':xgb}

In [11]:
def train_classifier(clf,x_train,y_train,x_test,y_test):
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    
    f1_train = f1_score(y_train,clf.predict(x_train),average="macro")

    f1_test = f1_score(y_test,y_pred,average="macro")
    
    return f1_train , f1_test

In [12]:
# promoted is 1 and not promoted is 0
# we are considering recall in this case because we may have two models with same f1-score 
# then we will select the one with better recall
# this is beacuse we dont want our model predict an employee as not promoted to an employee who is actually promoted .
# this will bring down the morale of loyal and hardworking employees
# the more that happens the more goes down the morale of hard working a loyal employees .

time.sleep(1)
start_ = time.time()

f1_scores_train = []
recall_scores_train = []
f1_scores_test = []
recall_scores_test = []

for name,clf in clfs.items():
    
    f1_train , f1_test= train_classifier(clf, x_train,y_train,x_test,y_test)
    
    print("For ",name)
    print("      f1_train ",f1_train)
    print("      f1_test ",f1_test)

    f1_scores_train.append(f1_train)
    f1_scores_test.append(f1_test)

    
print(f"total time taken to complete training fo all models is  : {(time.time()-start_)/60}")

For  KN
      f1_train  0.7622894024233281
      f1_test  0.6090990059314189
For  NB
      f1_train  0.5432953519264271
      f1_test  0.49577874867397387
For  DT
      f1_train  0.25111662582019095
      f1_test  0.22985122905544877
For  LR
      f1_train  0.7802037487032454
      f1_test  0.6724910976667895
For  RF
      f1_train  0.9958935649317998
      f1_test  0.6584792972739143
For  AdaBoost
      f1_train  0.40985622755728146
      f1_test  0.3782498220517271
For  Bc
      f1_train  0.9956662457630471
      f1_test  0.673482027658529
For  ETC
      f1_train  0.9960101038856762
      f1_test  0.6744502291309085
For  GBDT
      f1_train  0.7162186575849857
      f1_test  0.5749875571499595
For  xgb
      f1_train  0.9065630886817486
      f1_test  0.6711409847791895
total time taken to complete training fo all models is  : 200.27084197600684


In [13]:
# this is the performance considering region variable
performance_df = pd.DataFrame({'Algorithm':clfs.keys(),'f1_train':f1_scores_train,'f1_test':f1_scores_test}).sort_values('f1_test',ascending=False)

In [14]:
performance_df

,Algorithm,f1_train,f1_test
7,ETC,0.996010,0.674450
6,Bc,0.995666,0.673482
3,LR,0.780204,0.672491
9,xgb,0.906563,0.671141
4,RF,0.995894,0.658479
0,KN,0.762289,0.609099
8,GBDT,0.716219,0.574988
1,NB,0.543295,0.495779
5,AdaBoost,0.409856,0.378250
2,DT,0.251117,0.229851


# stacking

In [20]:

SGD_ = SGDClassifier(random_state=123, alpha=0.0001, loss='hinge', penalty='l2')
svm_model = SVC(random_state=123 ,C = 2 , kernel = 'rbf' , gamma = "scale") 

In [21]:
# Applying stacking
estimators=[('sgd_', SGD_), ('svm',svm_model)]
final_estimator= SVC(random_state=123 ,C = 2 , kernel = 'rbf' , gamma = "scale") 

In [22]:
from sklearn.ensemble import StackingClassifier

In [23]:
clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator)

In [24]:
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)


In [26]:
print("f1_train",f1_score(clf.predict(x_train),y_train,average='macro'))
print("f1_test",f1_score(y_test,y_pred,average='macro'))

f1_train 0.9529914285710743
f1_test 0.7442564295696775


# voting

In [ ]:
from sklearn.ensemble import VotingClassifier

SGD_ = SGDClassifier(random_state=123, alpha=0.0001, loss='hinge', penalty='l2')
svm_model = SVC(random_state=123 ,C = 2 , kernel = 'rbf' , gamma = "scale") 

In [ ]:
voting = VotingClassifier(estimators=[('sgd_', SGD_), ('svm',svm_model)],voting='soft')

In [ ]:
voting.fit(x_train,y_train)
y_pred = voting.predict(x_test)


In [ ]:
print("f1_train",f1_score(voting.predict(x_train),y_train))
print("f1_test",f1_score(y_test,y_pred))